# 100자 리뷰
item_id를 사용하여 100자리뷰 수집
- 사용파일: A_information_etc_utf8.csv
- 생성파일: A_100_reviews_utf8.csv

In [46]:
from bs4 import BeautifulSoup
from selenium import webdriver
import requests
import json
from selenium import webdriver
import time
import csv
import pandas as pd

In [69]:
# CSV 파일 경로
# 알라딘-총평점-v1.0 작업 이후 A_information_etc 파일로 작업필요
file_path = 'A_information_etc_utf8.csv'

# pandas를 사용하여 CSV 파일 읽기
df = pd.read_csv(file_path, encoding='utf-8')

# 'item_id' 열의 값들을 중복 없이 리스트로 변환
item_id_list = df['item_id'].unique().tolist()

In [70]:
len(item_id_list)

1023

In [77]:
reviews = []
reviews_html_list = []

In [78]:
pageCount=10000

In [81]:
count = 1
for item_id in item_id_list:
    #초기화
    review = {
                    'item_id' : None,
                    'paper_id': None,
                    'review_rating': None,
                    'review_like_count': None,
                    'created_at': None,
                    'review_comment': None,
                    'reviewer' : None
                }
    try:
        #디테일api로 접근 myreviewId필요
        Review100_url = f'https://www.aladin.co.kr/ucl/shop/product/ajax/GetCommunityListAjax.aspx?ProductItemId={item_id}&itemId={item_id}&pageCount={pageCount}&communitytype=CommentReview&nemoType=-1&page=1&startNumber=1&endNumber=1000&sort=2&IsOrderer=2&BranchType=1&IsAjax=true&pageType=0'
        #디테일 정보 얻기
        Review100_response = None
        Review100_response = requests.get(Review100_url)
        #디테일 정보 html 중 텍스트만 추출
        Review100_soup = BeautifulSoup(Review100_response.text, 'html.parser')
        all_hundred_list_divs = Review100_soup.find_all('div', class_='hundred_list')
        reviews_html = {
            'item_id' : item_id,
            'html': Review100_soup
            }
        reviews_html_list.append(reviews_html)
        print(count, time.strftime('%Y.%m.%d - %H:%M:%S'))
        count += 1
        for hundred_list_div in all_hundred_list_divs:
            paper_id = review_rating = agree_number = createAt = Review100detail = None
            try:
                ## 스코어계산
                #hl_star_div_list초기화
                specific_images=[]
                # "HL_star" 클래스를 가진 div 태그 찾기
                hl_star_div = hundred_list_div.find('div', class_='HL_star')
                specific_images = hl_star_div.find_all('img', src="//image.aladin.co.kr/img/shop/2018/icon_star_on.png")
                # 특정 이미지의 갯수 계산
                review_rating = len(specific_images)
            
                #공감수, 작성일 추출
                spans = hundred_list_div.find_all('span', class_='Ere_sub_gray8 Ere_fs13 Ere_PR10')
                agree_number = 0
                createAt = ""
                for span in spans:
                    if "(" in span.text:
                        agree_number = int(span.text.split('(')[1].split(')')[0])
                    elif  "-" in span.text:
                        createAt = span.text
                    else:
                        agree_number = 0
                #ID추출 - 하지말기
                paper_id_element = hundred_list_div.find('div', id=lambda x: x and x.startswith('spnPaper'))
                if paper_id_element['id'].split('r')[1] == 'Spoile':
                    paper_id = paper_id_element['id'].split("Spoiler")[1]
                else: 
                    paper_id = paper_id_element['id'].split("r")[1] 
                #리뷰내용
                Review100detail = hundred_list_div.find('div', id='spnPaper' + paper_id).text.strip()
                #리뷰어닉네임
                reviewer = hundred_list_div.find('a', class_='Ere_sub_gray8 Ere_fs13 Ere_PR10').text
        
                review = {
                    'item_id' : item_id,
                    'paper_id': paper_id,
                    'review_rating': review_rating,
                    'review_like_count': agree_number,
                    'created_at': createAt,
                    'review_comment': Review100detail,
                    'reviewer' : reviewer
                }
                reviews.append(review)
            except Exception as e:
                print(f'오류발생: {e} / item_id {item_id} paper_id {paper_id}')
                review = {
                    'item_id' : item_id,
                    'paper_id': paper_id,
                    'review_rating': review_rating,
                    'review_like_count': agree_number,
                    'created_at': createAt,
                    'review_comment': Review100detail,
                    'reviewer' : reviewer
                }

                reviews.append(review)
                continue
    except Exception as e:
        print(f'Unhandled exception for item_id {item_id}: {e}')
        continue  # 다음 item_id로 넘어갑니다

if len(reviews) >= 1:
    fields = list(reviews[0].keys())
    # 'encoding' 인자를 'utf-8'로 설정
    with open('A_100_reviews_utf8.csv', 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fields,escapechar='\\')
        writer.writeheader()
        writer.writerows(reviews)

1 2024.03.18 - 01:34:51
2 2024.03.18 - 01:34:54
3 2024.03.18 - 01:34:54
4 2024.03.18 - 01:34:55
5 2024.03.18 - 01:34:56
6 2024.03.18 - 01:34:56
7 2024.03.18 - 01:34:57
8 2024.03.18 - 01:34:58
9 2024.03.18 - 01:34:58
10 2024.03.18 - 01:34:59
11 2024.03.18 - 01:34:59
12 2024.03.18 - 01:35:00
13 2024.03.18 - 01:35:00
14 2024.03.18 - 01:35:02
15 2024.03.18 - 01:35:02
16 2024.03.18 - 01:35:03
17 2024.03.18 - 01:35:03
18 2024.03.18 - 01:35:04
19 2024.03.18 - 01:35:06
20 2024.03.18 - 01:35:06
21 2024.03.18 - 01:35:06
22 2024.03.18 - 01:35:07
23 2024.03.18 - 01:35:07
24 2024.03.18 - 01:35:08
25 2024.03.18 - 01:35:10
26 2024.03.18 - 01:35:11
27 2024.03.18 - 01:35:11
28 2024.03.18 - 01:35:12
29 2024.03.18 - 01:35:12
30 2024.03.18 - 01:35:13
31 2024.03.18 - 01:35:14
32 2024.03.18 - 01:35:14
33 2024.03.18 - 01:35:15
34 2024.03.18 - 01:35:15
35 2024.03.18 - 01:35:16
36 2024.03.18 - 01:35:16
37 2024.03.18 - 01:35:17
38 2024.03.18 - 01:35:17
39 2024.03.18 - 01:35:18
40 2024.03.18 - 01:35:19
41 2024.0

In [82]:
#누락된 item_id있는지 확인
missing_item_ids = set(item_id_list) - set([info['item_id'] for info in reviews])

if missing_item_ids:
    print("Missing item_ids:")
    for item_id in missing_item_ids:
        print(item_id)
else:
    print('누락된 아이디 없음')

#300262898, 269802093, 198016063 리뷰0건이라 없음

Missing item_ids:
300262898
269802093
198016063


In [83]:
#중복확인
df = pd.DataFrame(reviews)


df_clean1 = df.drop_duplicates(subset="paper_id", keep="first", ignore_index=True)
df_clean2 = df.drop_duplicates(subset="review_comment", keep="first", ignore_index=True)
df_clean3 = df.drop_duplicates(subset=["paper_id","review_comment"], keep="first", ignore_index=True)

print(len(df), len(df_clean1), len(df_clean2), len(df_clean3))
# # 결과 출력
# if len(duplicated_rows) >= 1:
#    print(duplicated_rows.sort_values(by='Review100detail'))
# else:
#     print('중복된 행이 없습니다')

94543 92903 88051 92903
         item_id  paper_id  review_rating  review_like_count  created_at  \
6203   239400216   7376312              5                  0  2015-02-11   
76795   26428054   7283603              4                  0  2014-12-16   
85119  193050475  11383214              5                  1  2019-12-23   
40350  301045967  13909468              5                  0  2022-09-06   
376     62809465   8042911              5                  0  2015-12-12   
...          ...       ...            ...                ...         ...   
65980  272354602  12651659              5                  2  2021-05-29   
65946  272354602  12677650              5                  2  2021-06-08   
65753  272354602  12679824              5                  4  2021-06-09   
65378  272354602  12659773              5                 11  2021-05-31   
65136  272354602  12648163              5                 69  2021-05-27   

      review_comment      reviewer  
6203                      

In [ ]:
#중복된 행 확인
duplicated_rows = df_clean1[df_clean1.duplicated('review_comment', keep=False)]
print(duplicated_rows.sort_values(by='review_comment'))
duplicated_rows1 = duplicated_rows.sort_values(by='review_comment')
duplicated_rows1.to_csv("중복확인용.csv")

In [85]:
#중복된 행 확인
duplicated_rows = df[df.duplicated('paper_id', keep=False)]
print(duplicated_rows.sort_values(by='review_comment'))
duplicated_rows1 = duplicated_rows.sort_values(by='paper_id')
duplicated_rows1.to_csv("중복확인용2.csv")

         item_id  paper_id  review_rating  review_like_count  created_at  \
6857   239400216   7376312              5                  0  2015-02-11   
78693   40869703   7376312              5                  0  2015-02-11   
56810  284657330  13365831              5                  2  2022-02-23   
69899  303025877  13365831              5                  2  2022-02-23   
69885  303025877  15020621              5                  2  2023-10-31   
...          ...       ...            ...                ...         ...   
6774   239400216   8394956              5                  1  2016-04-04   
713     62809465   7733206              3                  0  2015-08-24   
59      62809465   7733206              3                  0  2015-08-24   
19      62809465   7899360              5                  1  2015-11-07   
673     62809465   7899360              5                  1  2015-11-07   

                                          review_comment  reviewer  
6857              

In [86]:
#확인 후 저장
df_clean3.to_csv('A_100_reviews_utf8.csv', index=False, encoding='utf-8')